In [2]:
import xml.etree.ElementTree as ET
import xmltodict
import json
import os
import pandas as pd


data_cleaner_lookup = pd.read_csv('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/notebooks/eda/data_cleaning_requirement.csv')
data_cleaner_lookup.dropna(inplace=True)
data_cleaner_lookup = data_cleaner_lookup[['Annotation', 'Desired']].set_index('Annotation').to_dict()['Desired']

products = pd.read_csv('/home/sumit-tyagi/sumit-ssd/sumit/data/csvs/products.csv')
products.columns = ['id', 'brand_name', 'count', "#units_charged", 'loaded_weights']

def fun(x):
    x = x.lower()
    x = x.replace(" ", "_")
    x = x.replace("/", "_")
    return x
    
products.brand_name = products.brand_name.apply(fun)
product_name_list = products.brand_name.values.tolist()

annotations_to_be_removed = {'Complete Rack', "Strip", 'rack Row', 'rack row', 'strip', 'complete rack'}

annotation_path = r'/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/raw/annotations_master'
destination_path = os.path.join(annotation_path, 'cleaned')
os.mkdir(destination_path)

for i in os.walk(annotation_path):
    # print(i[2])
    files_effected = 0
    file_effected = False
    for j in i[2]:
        tree = ET.parse(os.path.join(annotation_path,j))
        xml_data = tree.getroot()
        for element in xml_data.findall('object'):

            if element.find('name').text in annotations_to_be_removed:
                xml_data.remove(element)
                file_effected = True

            if element.find('name').text not in product_name_list:
                file_effected = True
                if element.find('name').text in data_cleaner_lookup.keys():
                    element.find('name').text = data_cleaner_lookup[element.find('name').text]
                else:
                    element.find('name').text = 'other'

        if file_effected:
            files_effected += 1
            file_effected = False
    
        tree.write(os.path.join(destination_path, j))
    print("Total files effected: ", files_effected)


Total files effected:  1600
Total files effected:  1600


In [3]:
from collections import defaultdict


brand_names_count = defaultdict(int)

for i in os.walk('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/raw/annotations_master/cleaned'):
    # print(i)
    for j in i[2]:
        tree = ET.parse('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/raw/annotations_master/cleaned/'+j)
        xml_data = tree.getroot()
        xmlstr = ET.tostring(xml_data, encoding='utf-8', method='xml')

        data_dict = dict(xmltodict.parse(xmlstr))
        try:
            for i in data_dict['annotation']['object']:
                brand_names_count[i['name']] += 1
                i['name'] = 'packet'
        except:
            print("No objects in:", j)


import pandas as pd
brand_names_list = list(brand_names_count.keys())
annotation_data = pd.DataFrame(data = list(brand_names_count.items()), columns= ['brand_name', 'annotation_count'])


No objects in: PHOTO-2021-05-23-08-50-27.xml
No objects in: PHOTO-2021-07-05-12-23-19.xml
No objects in: PHOTO-2021-07-08-14-38-07.xml
No objects in: PHOTO-2021-07-01-09-23-18.xml
No objects in: PHOTO-2021-07-08-12-47-52.xml
No objects in: PHOTO-2021-07-10-08-24-01.xml
No objects in: PHOTO-2021-07-03-13-13-08.xml
No objects in: PHOTO-2021-06-24-10-31-22.xml
No objects in: PHOTO-2021-05-14-12-56-20.xml
No objects in: PHOTO-2021-05-29-15-57-27.xml
No objects in: PHOTO-2021-07-15-10-14-15.xml
No objects in: PHOTO-2021-07-26-09-43-37.xml


In [5]:
annotation_data.sort_values(by='brand_name')

,brand_name,annotation_count
18,arcoiris_clasico_paquetin,4528
1,arcoiris_fruts_paquetin,3590
20,barras_de_coco_paquetin,1393
30,bizcochitos_clasica_bolsa,849
22,bombitos_clasica_bolsa,1064
39,chocolatines_paquetin,38
32,chokis_brownie_paquetin,411
46,chokis_chocotella_paquetin,135
3,chokis_chokomax_paquetin,3350
38,chokis_clasico_paquetin,19


In [6]:
annotation_data[~annotation_data.brand_name.isin(products['brand_name'].values)]

,brand_name,annotation_count
5,emperador_nuez,3444
8,emperador_chocolate,7173
9,emperador_piruetas,4157
10,cremax_de_nieve_fresa,3090
11,cremax_de_nieve_chocolate,3245
12,cremax_de_nieve_vainilla,2916
14,marias_doradas,1404
16,other,3228
23,emperador_nocturno,1920
27,emperador_combinado,1683


In [7]:
products[~products.brand_name.isin(annotation_data['brand_name'].values)]

,id,brand_name,count,#units_charged,loaded_weights
4,20127,emperador_chocolate_paquetin,1287,72344,859446.72
9,20098,cremax_de_nieve_chocolate_paquetin,839,36708,436091.04
11,20124,emperador_piruetas_paquetin,684,30888,366949.44
17,20131,emperador_nuez_paquetin,414,18216,216406.08
18,20102,cremax_de_nieve_fresa_paquetin,415,17766,211060.08
19,20099,cremax_de_nieve_vainilla_paquetin,394,16968,201579.84
21,20106,cremax_de_nieve_chocolate_rollo,783,15623,304492.27
30,20319,marias_doradas_rollo,722,9440,183816.00
31,20132,emperador_chocolate_rollo,384,9024,175877.76
32,20130,emperador_combinado_paquetin,153,6776,80498.88
